In [1]:
import keras
import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Flatten, Dense, Input, Dropout
from keras.layers import Conv2D, ZeroPadding2D
from keras.layers import MaxPooling2D
from keras.optimizers import SGD
from keras import backend as K

Using TensorFlow backend.


In [2]:
steps_per_epoch=50
epochs=50

In [3]:
img_width = 382
img_height = 345
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [4]:
def VGG11(input_shape, num_classes):
    
    if K.image_data_format() == 'channels_first':
        input_shape = input_shape[::-1]
    else:
        input_shape = input_shape

    model = Sequential()

    # Block 1
    model.add(ZeroPadding2D((1, 1), input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu', name='block1_conv1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu', name='block1_conv2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))
    
    # Block 2
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, (3, 3), activation='relu', name='block2_conv1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, (3, 3), activation='relu', name='block2_conv2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))
    
    # Block 3
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu', name='block3_conv1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu', name='block3_conv2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))
    
    # Block 4
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu', name='block4_conv1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu', name='block4_conv2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))
     
    # Block 5
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu', name='block5_conv1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu', name='block5_conv2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

In [5]:
train_datagen = ImageDataGenerator(
        samplewise_center=True,
        samplewise_std_normalization=True,
        horizontal_flip=True,
        vertical_flip=True)

In [6]:
test_datagen = ImageDataGenerator()

In [7]:
train_generator = train_datagen.flow_from_directory(
    '../data/data_small/',     
    class_mode='categorical',           
    target_size=(224, 224),
    batch_size=10)

validation_generator = test_datagen.flow_from_directory(
    '../data/data_small/',  
    class_mode='categorical',
    target_size=(224, 224),
    batch_size=10)

Found 1970 images belonging to 15 classes.
Found 1970 images belonging to 15 classes.


In [8]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=True)

In [9]:
model = VGG11((224, 224, 3), 15)

In [10]:
sgd = SGD(lr=0.01, decay=0.0005, momentum=0.9) # nestrov=True ??
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [1]:
model.fit_generator(
    generator=train_generator,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=200,
    callbacks=[tbCallBack],
    steps_per_epoch=200,
    verbose=2)

In [ ]:
model.save_weights('my_model_weights.h5')